# Global Stuffs

In [1]:
# use env bl17_2 to run this script

import param
import panel as pn

import holoviews as hv
import imageio
import os, glob, time

import json
import pandas as pd

import tqdm
import numpy as np

In [5]:
from tkinter import font
import numpy as np
import matplotlib.pyplot as plt
import pyFAI

def plot_2d_images(image, poni_file,output_folder,tif_file,vmin=None, vmax=None):

    # Load poni file to extract calibration parameters
    ai = pyFAI.load(poni_file)
    pixel_size = ai.pixel1  # Assuming square pixels, you can also use ai.pixel2
    detector_distance = ai.dist
    beam_x = ai.poni1 / ai.pixel1
    beam_y = ai.poni2 / ai.pixel2
    wavelength = ai.wavelength

    # Convert SAXS to q-space coordinates
    x_pixels = image.shape[1]
    y_pixels = image.shape[0]
    x_coords = np.arange(x_pixels) - beam_x
    y_coords = np.arange(y_pixels) - beam_y
    xx, yy = np.meshgrid(x_coords, y_coords)

    qx = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * xx / detector_distance)
    qy = 1e-9 * 2 * np.pi / wavelength * np.sin(pixel_size * yy / detector_distance)

    # Determine tick values based on qx and qy range
    qx_min, qx_max = np.min(qx), np.max(qx)
    qy_min, qy_max = np.min(qy), np.max(qy)
    #qx_ticks = np.linspace(qx_min, qx_max, num=4)  # You can adjust the number of ticks here
    #qy_ticks = np.linspace(qy_min, qy_max, num=4)  # You can adjust the number of ticks here
    
    # Plot and save image
    fig, ax = plt.subplots(figsize=(6, 6))
    
    im_avg = ax.pcolormesh(qx, qy, np.log10(image), cmap='jet', vmin=vmin, vmax=vmax)
    ax.set_aspect('equal')
    cbar_avg = fig.colorbar(im_avg, ax=ax, shrink=.8, label='Intensity [a.u.]')
    cbar_avg.ax.tick_params(labelsize=20)
    # fontsize of intesity label to 20
    cbar_avg.ax.yaxis.label.set_size(20)
    #cbar_avg.ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{10**x:.0e}"))
    ax.set_xlabel(r"$q_x$ [nm$^{-1}$]", fontsize=20)
    ax.set_ylabel(r"$q_y$ [nm$^{-1}$]", fontsize=20)
    ax.set_title(f"{f"{os.path.splitext(tif_file)[0]}_2D"}", fontsize=14, y=1.05)
    #ax.set_xticks(qx_ticks)
    #ax.set_yticks(qy_ticks)
    ax.tick_params(axis='both', which='major', labelsize=20, width=1.5)

    # Save and show the figure
    plt.savefig(f"{output_folder}/{f"{os.path.splitext(tif_file)[0]}"}.png", dpi=300, bbox_inches='tight')
    #plt.show()
    plt.close()

def sort_files(filenames):
    def extract_numbers(filename):
        match = re.search(r'_s(\d+)_0*(\d+)', filename)
        if match:
            snumber = int(match.group(1))
            number = int(match.group(2))
            return snumber, number
        return float('inf'), float('inf')  # Return large values if the pattern doesn't match

    return sorted(filenames, key=extract_numbers)

def extract_metadata(metadata):
    # Extract the SPEC data
    S = metadata['SPEC']['S']
    i0 = S['1'] if '1' in S else None
    i1 = S['2'] if '2' in S else None
    Photod = S['4'] if '4' in S else None

    # Extract the series_date
    real_date_time = metadata['Eiger_metadata']['series_date']

    return real_date_time, i0, i1, Photod

#i0_1, i1_1, Photod_1 = extract_values(metadata_1)
#i0_2, i1_2, Photod_2 = extract_values(metadata_2)



# azm and rad integration function: imageio

In [12]:
import os
import glob
import pyFAI
import numpy as np
import matplotlib.pyplot as plt
import imageio
import fabio
import h5py
import json
from tqdm import tqdm
import pandas as pd
import re



def integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=3, q1=None, q2=None, vmin=None, vmax=None):
    # Locate all 'Threshold' folders
    threshold_folders = glob.glob(os.path.join(base_path, samp_folder, '*', 'Threshold*'))
    
    if not threshold_folders:
        raise FileNotFoundError("No Threshold folders found.")
    
    # Initialize the PyFAI azimuthal integrator
    ai = pyFAI.load(poni_file)

    # Read the mask file with fabio
    mask = fabio.open(mask_file).data

    # Loop through each threshold folder
    for threshold_folder in threshold_folders:
        intermediate_folder = os.path.basename(os.path.dirname(threshold_folder))
        output_folder = os.path.join(output_base_path, 'D1', samp_folder, intermediate_folder)

        # Ensure the output folder exists
        os.makedirs(output_folder, exist_ok=True)

        # List all .tif files in the current threshold folder
        tif_files = [f for f in os.listdir(threshold_folder) if f.endswith('.tif')]
        tif_files = sort_files(tif_files)

        if not tif_files:
            print(f"No .tif files found in {threshold_folder}.")
            continue

        # Initialize data containers for plotting
        azimuthal_data = []
        radial_data = []

        # Use tqdm for a progress bar with a shorter description
        for i, tif_file in enumerate(tqdm(tif_files, desc='Processing', unit='file'), start=1):

            file_path = os.path.join(threshold_folder, tif_file)
            img = imageio.v3.imread(file_path)
            
            an_img = imageio.v3.immeta(file_path)
            eiger_metadata = json.loads(an_img['description'])
            #print(f"eiger_metadata: {eiger_metadata}")
            

            real_date_time, i0, i1, Photod = extract_metadata(eiger_metadata)
            normfactor = Photod #/ i1
            #print(f'real_date_time: {real_date_time}, i0: {i0}, i1: {i1}, Photod: {Photod}, normfactor: {normfactor}')
            #print(f"i1: {i1}, Photod: {Photod}, normfactor: {normfactor}")

            if i % nsave2d == 0:
                plot_2d_images(img, poni_file, output_folder, tif_file, vmin=vmin, vmax=vmax)

            # Azimuthal integration
            result = ai.integrate1d(img, 1000, error_model='poisson', normalization_factor=normfactor, mask=mask)
            q, I_azimuthal, error_azm = result.radial, result.intensity, result.sigma
            azimuthal_data.append((q, I_azimuthal, error_azm, tif_file, real_date_time, i0, i1, Photod))

            # Radial integration
            chi, I_radial = ai.integrate_radial(
                img,
                npt=720,
                npt_rad=100,
                radial_range=(q1, q2), 
                azimuth_range=(-180, 180),
                mask=mask,
                normalization_factor=normfactor,
                method='csr',
                unit='chi_deg',
                radial_unit='q_nm^-1'
            )
            radial_data.append((chi, I_radial, tif_file, real_date_time, i0, i1, Photod))

        # Save the azimuthal data in HDF5 format
        output_folder_basename = os.path.basename(output_folder)
        with h5py.File(os.path.join(output_folder, f'{output_folder_basename}_azimuthal_data.h5'), 'w') as hf:
            for idx, (q, I, err, tif_file, real_date_time, i0, i1, Photod) in enumerate(azimuthal_data):
                group = hf.create_group(f'data_{idx}')
                group.create_dataset('q', data=q)
                group.create_dataset('I', data=I)
                group.create_dataset('error', data=err)
                group.create_dataset('filename', data=np.bytes_(tif_file))
                group.create_dataset('real_date_time', data=np.bytes_(real_date_time))
                group.create_dataset('i0', data=np.float32(i0) if i0 is not None else np.nan)
                group.create_dataset('i1', data=np.float32(i1) if i1 is not None else np.nan)
                group.create_dataset('Photod', data=np.float32(Photod) if Photod is not None else np.nan)

        # Save the radial data in HDF5 format
        output_folder_basename = os.path.basename(output_folder)
        with h5py.File(os.path.join(output_folder, f'{output_folder_basename}_radial_data.h5'), 'w') as hf:
            for idx, (chi, I, tif_file, real_date_time, i0, i1, Photod) in enumerate(radial_data):
                group = hf.create_group(f'data_{idx}')
                group.create_dataset('chi', data=chi)
                group.create_dataset('I', data=I)
                group.create_dataset('filename', data=np.bytes_(tif_file))
                group.create_dataset('real_date_time', data=np.bytes_(real_date_time))
                group.create_dataset('i0', data=np.float32(i0) if i0 is not None else np.nan)
                group.create_dataset('i1', data=np.float32(i1) if i1 is not None else np.nan)
                group.create_dataset('Photod', data=np.float32(Photod) if Photod is not None else np.nan)

        # Plot the integrated data
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 8))

        # Plot azimuthal data
        for q, I, err, tif_file, *_ in azimuthal_data:
            ax1.plot(q, I, label=tif_file)
            if q1 is not None:
                ax1.axvline(x=q1, color='r', linestyle='--')
            if q2 is not None:
                ax1.axvline(x=q2, color='r', linestyle='--')

        ax1.set_xlabel('q [1/nm]')
        ax1.set_ylabel('Intensity [a.u.]')
        ax1.set_title('Azimuthal Integration')
        #ax1.legend()

        # Plot radial data
        for chi, I, tif_file, *_ in radial_data:
            ax2.plot(chi, I, 'o', markersize=2, label=tif_file)
        ax2.set_xlabel('chi [degrees]')
        ax2.set_ylabel('Intensity [a.u.]')
        ax2.set_title('Radial Integration')
        #ax2.legend()

        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, 'integrated_plot.png'))
        plt.close(fig)

# in-situ

## LDPE, PHPD and P5HV

In [ ]:
import os

# Base paths and file definitions
base_path = '/Users/akmaurya/Desktop/20240628/in_situ'
poni_file = os.path.join(base_path, 'poni/LaB6_insituT.poni')
mask_file = os.path.join(base_path, 'poni/insituT_mask_LDPE.edf')
output_base_path = base_path  # Base path for the output directory structure

# q-value range
q1 = 14  # Minimum q-value (1/nm)
q2 = 16  # Maximum q-value (1/nm)
vmin, vmax = 1.5, 3  # Define vmin and vmax as needed

# List of sample folders
folders = [
    "LDPE/LDPE_film_grade_07",
    "PHPD/PHPD_10",
    "P5HV/P5HV_09",
    "P5HV/P5HV_0_Rot_phi_78",
    "P5HV/P5HV_1000_Rot_phi_81",
    "P5HV/P5HV_300_Rot_phi_79",
    "P5HV/P5HV_550_Rot_phi_80"
]

for folder in folders:
    integrate_tif_files(base_path, poni_file, mask_file, folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=vmin, vmax=vmax)


## P3HB

In [ ]:
q1, q2 = 8, 11
vmin, vmax = 1, 2.5

# List of P3HB sample folders
folders = [
    "P3HB/P3HB_B7i3_14",
    "P3HB/P3HB_B5i5_15",
    "P3HB/P3HB_B3i7_16"
]

for folder in folders:
    samp_folder = folder.replace("P3HB/", "polycond/")
    # Call the function with the updated folder path and parameters
    integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=vmin, vmax=vmax)


## Polycondensation polymer

In [ ]:
# polycond 

q1 = 14  # Minimum q-value (1/nm)
q2 = 16.0  # Maximum q-value (1/nm)
samp_folder = "polycond/PESt_C18_A_68"
integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=1.5, vmax=3)
samp_folder = "polycond/PESt_C18_NA_63"
integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=1.5, vmax=3) 


# Ex-situ

## LDPE, PHPD and P5HV

In [ ]:
# Define base paths and parameters
base_path = '/Users/akmaurya/Desktop/20240628/ex_situ'
poni_file = os.path.join(base_path, 'poni/LaB6_exsitu.poni')
mask_file = os.path.join(base_path, 'poni/insituT_mask_LDPE.edf')
output_base_path = base_path  # Base path for the output directory structure

q1 = 14  # Minimum q-value (1/nm)
q2 = 16.0  # Maximum q-value (1/nm)
vmin, vmax = 1, 2.5  # Define vmin and vmax as needed

# List of sample folders
folders = [
    "LDPE/LDPE_Film_ExSitu_37",
    "PHPD/PHPD_ExSitu_27",
    "P5HV/PHPD_MeltBlown_ExSitu_28",
    "P5HV/P5HV_ExSitu_26"
]

for folder in folders:
    samp_folder = f"{folder}"
    # Call the function with the updated folder path and parameters
    integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=vmin, vmax=vmax)


## fibers

In [13]:
q1 = 10  # Minimum q-value (1/nm)
q2 = 20  # Maximum q-value (1/nm)

samp_folder = "fibers/fiber_bundle_PET_25"
integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2,vmin=1, vmax=2.5)

q1 = 7  # Minimum q-value (1/nm)
q2 = 17  # Maximum q-value (1/nm)
samp_folder = "fibers/PHA_3B_Fiber_31"
integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2,vmin=1, vmax=2.5)

Processing:  48%|████▊     | 29/60 [00:04<00:03,  8.06file/s]/var/folders/0q/m0d3p44n2cb5dg956r2m2f782yh1mf/T/ipykernel_21829/8794536.py:35: RuntimeWarning: divide by zero encountered in log10
  im_avg = ax.pcolormesh(qx, qy, np.log10(image), cmap='jet', vmin=vmin, vmax=vmax)
Processing:  66%|██████▌   | 119/180 [00:24<00:12,  4.89file/s]


FileNotFoundError: [Errno 2] No such file or directory: '/Users/akmaurya/Desktop/20240628/ex_situ/D1/fibers/PHA_3B_Fiber_31/PHA_3B_Fiber_31_Scan00003/thresh1_PHA_3B_Fiber_31_Scan00003_s00003_00029.png'

Error in callback <function flush_figures at 0x15eaac9a0> (for post_execute), with arguments args (),kwargs {}:


KeyboardInterrupt: 

## P3HB

In [ ]:
# Define parameters
q1, q2 = 8, 11
vmin, vmax = 1, 2.5

folders = [
    "P3HB/P3HB_7to3_ExSitu_Film_29",
    "P3HB/P3HB_7to3_Chunk_ExSitu_32",
    "P3HB/P3HB_5to5_Film_ExSitu_33",
    "P3HB/P3HB_5to5_Chunk_ExSitu_34",
    "P3HB/P3HB_3to7_Film_ExSitu_35",
    "P3HB/P3HB_3to7_Chunk_ExSitu_36"
]

for folder in folders:
    
    samp_folder = f"{folder}"
    integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=vmin, vmax=vmax)


## Polycondensation polymer


In [ ]:
q1, q2 = 14, 16  # Define q1 and q2 as needed
vmin, vmax = 1, 2.5  # Define vmin and vmax as needed

folders = [
    "PEDO_C12_A_Water24hr_70",
    "PESt_C18_A_Water24hr_77",
    "PEDO_C12_NA_Water24hr_74",
    "PESt_C18_Ann_ExSitu_41",
    "PEDo_C12_Ann_ExSitu_40",
    "PESt_C18_Ann_Water20min_49",
    "PEDo_C12_Ann_Water20min_48",
    "PESt_C18_NA_Water24hr_73",
    "PEDo_C12_NotAnn_ExSitu_44",
    "PESt_C18_NotAnn_ExSitu_45",
    "PEDo_C12_NotAnn_Water20m_52",
    "PESt_C18_NotAnn_Water20m_53",
    "PESb_C10_A_Water24hr_71",
    "PESu_C4_A_Water24hr_72",
    "PESb_C10_Ann_ExSitu_39",
    "PESu_C4_Ann_ExSitu_38",
    "PESb_C10_Ann_Water20min_47",
    "PESu_C4_Ann_Water20min_46",
    "PESb_C10_NA_Water24hr_75",
    "PESu_C4_NA_Water24hr_76",
    "PESb_C10_NotAnn_ExSitu_43",
    "PESu_C4_NotAnn_ExSitu_42",
    "PESb_C10_NotAnn_Water20m_51",
    "PESu_C4_NotAnn_Water20m_50",
    "PESt_C18_A_Water24hr_69"
]

for folder in folders:
    samp_folder = f"polycond/{folder}"
    integrate_tif_files(base_path, poni_file, mask_file, samp_folder, output_base_path, nsave2d=30, q1=q1, q2=q2, vmin=vmin, vmax=vmax)
